## Solve Model balancing problem with kinetic parameters as variables

In [1]:
import numpy as np
from model_balancing import ModelBalancing
from util import Q_

In [2]:
use_regulation = True
args = {}

# choose solver

args["solver"] = "SLSQP"

In [3]:
# Network topology

args["S"] = np.matrix(
    [
        [-1, 0, 0],
        [1, -1, -1],
        [0, 1, 1]
    ], dtype=float
)

if use_regulation:
    args["A_act"] = np.matrix(
        [
            [0, 0, 1],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )
    args["A_inh"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )
else:
    args["A_act"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )
    args["A_inh"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )

In [4]:
# Priors for Kinetic and Thermodynamic parameters

args["Keq_gmean"] = Q_([1, 1, 1], "")  # geometric mean (assuming a standard concentration of 1M)
args["Keq_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
)  # log-scale covariance

args["kcatf_gmean"] = Q_([20.0, 8.0, 10.0], "1/s")  # geometric mean
args["kcatf_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
) # log-scale covariance

args["kcatr_gmean"] = Q_([3.5e-3, 6.2e4, 3.1e-1], "1/s")  # geometric mean
args["kcatr_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
) # log-scale covariance

# the K parameters are only for existing connections in the S, A_act and A_inh matrices.
# the order of values is "metabolite-first".
args["Km_gmean"] = Q_([1e-2, 1e-4, 1e-4, 1e-3, 1e-1, 1e-1], "M")
args["Km_ln_cov"] = np.array(
    [
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ]
) # log-scale covariance

args["Ka_gmean"] = Q_([1e-3, 5e-3], "M")
args["Ka_ln_cov"] = np.array(
    [
        [1, 0],
        [0, 1],
    ]
)

args["Ki_gmean"] = Q_([1e-3], "M")
args["Ki_ln_cov"] = np.array(
    [
        [1],
    ]
)

In [5]:
# condition dependent data (columns represent conditions)

args["fluxes"] = Q_(
    [
        [2.0, 1.5, 2.5, 2.5],
        [1.0, 1.0, 2.0, 0.5],
        [1.0, 0.5, 0.5, 2.0],
    ], "mM/s"
)

args["conc_enz_gmean"] = Q_(
    [
        [1e-3, 1e-3, 1e-3, 3e-3],
        [2e-3, 3e-3, 1e-3, 2e-3],
        [1e-3, 2e-3, 2e-3, 1e-3],
    ], "M")  # geometric mean

args["conc_enz_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

args["conc_met_gmean"] = Q_(
    [
        [3e-3, 4e-3, 9e-4, 3e-4],
        [2e-3, 3e-3, 6e-4, 2e-4],
        [1e-3, 2e-3, 3e-4, 1e-4],
    ], "M"
)  # geometric mean

args["conc_met_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

In [6]:
args["rate_law"] = "CM"

mb = ModelBalancing(**args)

# check that the driving forces at the mode are all positive
assert mb.is_gmean_feasible()

mb.initialize_with_gmeans()
print(f"initial total squared Z-scores = {mb.objective_value:.3f}")

mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")

print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores(5)

mb.print_status()

initial total squared Z-scores = 0.000
optimized total squared Z-scores = 0.000
------------------------- All Z-scores -------------------------
Km = 0.0
Ka = 0.0
Ki = 0.0
Keq = 0.0
kcatf = 0.0
conc_met = 0.0
kcatr = 0.0
conc_enz = 0.0

Metabolite concentrations (M) =
 [[3.e-03 4.e-03 9.e-04 3.e-04]
 [2.e-03 3.e-03 6.e-04 2.e-04]
 [1.e-03 2.e-03 3.e-04 1.e-04]]

Enzyme concentrations (M) =
 [[0.0213     0.02355    0.02954167 0.037875  ]
 [0.00026263 0.00038775 0.00058358 0.00018756]
 [0.00040133 0.00025125 0.00056402 0.01040867]]

Driving forces (RT) =
 [[0.40546511 0.28768207 0.40546511 0.40546511]
 [0.69314718 0.40546511 0.69314718 0.69314718]
 [0.69314718 0.40546511 0.69314718 0.69314718]]

η(thr) =
 [[0.33 0.25 0.33 0.33]
 [0.5  0.33 0.5  0.5 ]
 [0.5  0.33 0.5  0.5 ]]

η(kin) =
 [[0.01 0.01 0.01 0.01]
 [0.95 0.97 0.86 0.67]
 [0.66 0.75 0.37 0.17]]

η(reg) =
 [[1.   1.   1.   1.  ]
 [1.   1.   1.   1.  ]
 [0.75 0.8  0.47 0.23]]




